In [163]:
# window_size_ma=20
# threshold_param=100
# long_probability_threshold=0.55
# short_probability_threshold=0.55
# fees = 0.0015 
# stop_loss = 0.01
# sl_trail = 0.02
# entry1=0.7
# entry2=0.55
# entry3=0.15
# entry4=0.05
# # position_sizes = {
# #     20: 0.02,  
# #     50: 0.25,  
# #     70: 0.85   
# # }

In [177]:
window_size_ma=20
threshold_param=100
long_probability_threshold=0.55
short_probability_threshold=0.55
fees = 0.0015 
stop_loss = 0.01
sl_trail = 0.05
entry1=0.7
entry2=0.55
entry3=0.15
entry4=0.05
# position_sizes = {
#     20: 0.02,  
#     50: 0.25,  
#     70: 0.85   
# }

In [178]:
import pandas as pd
import vectorbt as vbt
import numpy as np
data = pd.read_csv('btc_1h_test.csv')
column_names = ['datetime', 'open', 'high', 'low', 'close', 'volume']
data.columns = column_names

data['datetime'] = pd.to_datetime(data['datetime'])
# data.set_index('datetime', inplace=True)
data

,datetime,open,high,low,close,volume
0,2021-02-01 01:30:00,32807.29,32931.55,32461.23,32853.73,2968.315197
1,2021-02-01 02:30:00,32853.73,32899.39,32515.00,32560.73,2336.553165
2,2021-02-01 03:30:00,32561.35,33088.88,32559.85,32974.10,1874.807965
3,2021-02-01 04:30:00,32974.10,33255.49,32933.54,33092.98,3218.738903
4,2021-02-01 05:30:00,33092.97,33106.33,32296.16,32546.27,4383.926122
...,...,...,...,...,...,...
8723,2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.922560
8724,2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.060520
8725,2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.658550
8726,2022-01-31 04:30:00,37785.78,38230.00,37778.46,37881.76,1728.811800


In [179]:
nan_values = data.isna().sum()
nan_values

datetime    0
open        0
high        0
low         0
close       0
volume      0
dtype: int64

In [180]:
data['Price Change'] = data['close'].diff()
data['Gain'] = data['Price Change'].apply(lambda x: x if x > 0 else 0).rolling(window=14).mean()
data['Loss'] = -data['Price Change'].apply(lambda x: x if x < 0 else 0).rolling(window=14).mean()
data['RS'] = data['Gain'] / data['Loss']
data['rsi'] = 100 - (100 / (1 + data['RS']))
data = data.drop(['Price Change', 'Gain', 'Loss', 'RS'], axis=1)

data["short_mom"] = data["rsi"].rolling(window=10, min_periods=1, center=False).mean()
data["long_mom"] = data["rsi"].rolling(window=20, min_periods=1, center=False).mean()
data["short_mov"] = data["close"].rolling(window=10, min_periods=1, center=False).mean()
data["long_mov"] = data["close"].rolling(window=20, min_periods=1, center=False).mean()
data.tail()

,datetime,open,high,low,close,volume,rsi,short_mom,long_mom,short_mov,long_mov
8723,2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.92256,40.961362,45.235850,50.693304,37867.200,37956.4560
8724,2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.06052,38.211991,44.710243,49.914299,37829.098,37946.3185
8725,2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.65855,44.645517,44.718328,49.541869,37813.829,37938.0175
8726,2022-01-31 04:30:00,37785.78,38230.00,37778.46,37881.76,1728.81180,49.805624,45.248679,49.808902,37805.683,37936.2490
8727,2022-01-31 05:30:00,37881.75,37951.07,36828.32,36829.30,2907.71096,25.293105,42.708663,47.925771,37704.714,37872.7470


In [181]:
data['label']=np.where(data.short_mov>data.long_mov,1,0)
data.tail()

,datetime,open,high,low,close,volume,rsi,short_mom,long_mom,short_mov,long_mov,label
8723,2022-01-31 01:30:00,37512.37,37740.00,37351.63,37684.45,1037.92256,40.961362,45.235850,50.693304,37867.200,37956.4560,0
8724,2022-01-31 02:30:00,37684.45,37762.60,37492.25,37729.00,874.06052,38.211991,44.710243,49.914299,37829.098,37946.3185,0
8725,2022-01-31 03:30:00,37729.00,37855.54,37625.41,37785.78,734.65855,44.645517,44.718328,49.541869,37813.829,37938.0175,0
8726,2022-01-31 04:30:00,37785.78,38230.00,37778.46,37881.76,1728.81180,49.805624,45.248679,49.808902,37805.683,37936.2490,0
8727,2022-01-31 05:30:00,37881.75,37951.07,36828.32,36829.30,2907.71096,25.293105,42.708663,47.925771,37704.714,37872.7470,0


In [182]:
from joblib import load

loaded_clf = load('saved_rf_model.joblib')


In [183]:
from sklearn.preprocessing import StandardScaler
from ta.volatility import AverageTrueRange

data['MA'] = data['close'].rolling(window=window_size_ma).mean()

data['price_diff'] = data['close'].diff()
data['target'] = (data['price_diff'] > threshold_param).astype(int)

data['ATR'] = AverageTrueRange(data['high'], data['low'], data['close'], window=14).average_true_range()

for lag in range(1, 5):
    data[f'lag_{lag}_ATR'] = data['ATR'].shift(lag)

data.dropna(inplace=True)


X_new = data[['MA', 'ATR', 'lag_1_ATR', 'lag_2_ATR', 'lag_3_ATR', 'lag_4_ATR']]

scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new)


predictions = loaded_clf.predict(X_new_scaled)  

data['predictions'] = predictions

In [184]:
price_increase_prob = loaded_clf.predict_proba(X_new_scaled)[:, 1]
price_decrease_prob = loaded_clf.predict_proba(X_new_scaled)[:, 0]

data['entry_long'] = (
    (data['label'] == 1) & (price_increase_prob >= long_probability_threshold)
).astype(int)

data['exit_long'] = (
    (data['label'] == 0) & (price_decrease_prob >= short_probability_threshold)
).astype(int)


In [185]:
num_entry_long = data['entry_long'].sum()
num_exit_long = data['exit_long'].sum()
print(num_entry_long)
print(num_exit_long)

215
3466


In [186]:
value_counts = data['predictions'].value_counts()
print("Unique values:", value_counts.index.tolist())
print("Value counts:\n", value_counts)

Unique values: [0, 1]
Value counts:
 predictions
0    7864
1     845
Name: count, dtype: int64


In [187]:
data['position_size'] = np.where(
    (data['predictions'] == 1) & (price_increase_prob > 0.85),
    entry1,  
    np.where(
        (data['predictions'] == 1) & (price_increase_prob > 0.75),
        entry2,  
        np.where(
            (data['predictions'] == 1) & (price_increase_prob > 0.65),
            entry3,  
            np.where(
                (data['predictions'] == 1) & (price_increase_prob > 0.55),
                entry4,  
                0.0   
            )
        )
    )
)

# Check the distribution of position sizes
value_counts = data['position_size'].value_counts()
print("Unique values:", value_counts.index.tolist())
print("Value counts:\n", value_counts)


Unique values: [0.0, 0.05, 0.15, 0.55]
Value counts:
 position_size
0.00    8246
0.05     396
0.15      63
0.55       4
Name: count, dtype: int64


In [188]:
portfolio = vbt.Portfolio.from_signals(
    close=data['close'], 
    entries=data['entry_long'],  
    exits=data['exit_long'],
    size=data['position_size'],  
    size_type="Percent",
    fees=fees,  
    sl_stop=stop_loss,
    # tp_stop=take_profit,
    sl_trail=sl_trail ,
    init_cash=100000
)

portfolio_stats = portfolio.stats()
portfolio_stats

/home/outbreakkp/anaconda3/envs/vectorbt_env/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/home/outbreakkp/anaconda3/envs/vectorbt_env/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/home/outbreakkp/anaconda3/envs/vectorbt_env/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/home/outbreakkp/anaconda3/envs/vectorbt_env/lib/python3.8/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                             19.000000
End                             8727.000000
Period                          8709.000000
Start Value                   100000.000000
End Value                     100089.737898
Total Return [%]                   0.089738
Benchmark Return [%]              11.064871
Max Gross Exposure [%]            56.009416
Total Fees Paid                 2342.885894
Max Drawdown [%]                   2.004134
Max Drawdown Duration           5559.000000
Total Trades                     119.000000
Total Closed Trades              119.000000
Total Open Trades                  0.000000
Open Trade PnL                     0.000000
Win Rate [%]                      27.731092
Best Trade [%]                     7.855536
Worst Trade [%]                   -4.609093
Avg Winning Trade [%]              2.152443
Avg Losing Trade [%]              -1.197056
Avg Winning Trade Duration         8.696970
Avg Losing Trade Duration          4.430233
Profit Factor                   

In [189]:
portfolio.plot().show()